<H1>ФБ-25 Слобода Ірина</H1>
<h2>Лабораторна робота 2</h2>

In [112]:
import datetime
import os
import urllib
from prettytable import PrettyTable
import pandas as pd

print("Setup ok")

Setup ok


Функція для завантаження даних

In [110]:
def download_data(province_id, year1=1981, year2=2024):
    # Checking the existence of the data folder
    data_folder = 'data'
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    else:
        # Checking the existence of an already downloaded file
        existing_files = [f for f in os.listdir(data_folder) if f.startswith(f'vhi_id__{province_id}__')]
        if existing_files:
            print(f"File '{existing_files[0]}' already exists in the 'data' folder. Nothing to download.")
            return

    # Loading data
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year1}&year2={year2}&type=Mean"
    vhi_url = urllib.request.urlopen(url)

    # Writing to a file
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    filename = f'vhi_id__{province_id}__{current_datetime}.csv'
    file_path = os.path.join(data_folder, filename)

    with open(file_path, 'wb') as out:
        out.write(vhi_url.read())

    print(f"VHI is downloaded and saved in '{file_path}'")

Завантаження тестових структуровних данних за номером області

In [111]:
for i in range(1,28):
  download_data(i)

File 'vhi_id__1__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__2__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__3__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__4__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__5__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__6__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__7__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__8__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__9__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__10__2024-05-28_15-49.csv' already exists in the 'data' folder. Nothing to download.
File 'vhi_id__11__2024-05-28_

Відповідність id регіону та його назви

In [97]:
reg_id_name = {
    1: 'Вінницька',  2: 'Волинська',  3: 'Дніпропетровська',  4: 'Донецька',  5: 'Житомирська',
    6: 'Закарпатська',  7: 'Запорізька',  8: 'Івано-Франківська',  9: 'Київська',  10: 'Кіровоградська',
    11: 'Луганська',  12: 'Львівська',  13: 'Миколаївська',  14: 'Одеська',  15: 'Полтавська',
    16: 'Рівенська',  17: 'Сумська',  18: 'Тернопільська',  19: 'Харківська',  20: 'Херсонська',
    21: 'Хмельницька',  22: 'Черкаська',  23: 'Чернівецька',  24: 'Чернігівська',  25: 'Республіка Крим'
}

Функція очищення даних і створення загального DataFrame


In [103]:
def create_data_frame(folder_path):

    csv_files = glob.glob(folder_path + "/*.csv")

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    frames = []

    for file in csv_files:
        region_id = int(file.split('__')[1])
        df = pd.read_csv(file, header = 1, names = headers)
        df.at[0, 'Year'] =  df.at[0, 'Year'][9:]
        df=df.drop(df.index[-1])
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df.insert(0, 'region_id', region_id, True)
        df['Week'] = df['Week'].astype(int)
        
        frames.append(df)   
    result = pd.concat(frames).drop_duplicates().reset_index(drop=True)

    result = result.loc[(result.region_id != 12) & (result.region_id != 20)]
    result = result.replace({'region_id':{1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21,
                                          11:9, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 21:17, 
                                          22:18, 23:6, 24:1, 25:2, 26:6, 27:5}})
    return result

In [105]:
resultdf = create_data_frame('data')

print(resultdf)

       region_id  Year  Week    SMN     SMT    VCI    TCI    VHI
0             21  1982     1  0.059  258.24  51.11  48.78  49.95
1             21  1982     2  0.063  261.53  55.89  38.20  47.04
2             21  1982     3  0.063  263.45  57.30  32.69  44.99
3             21  1982     4  0.061  265.10  53.96  28.62  41.29
4             21  1982     5  0.058  266.42  46.87  28.57  37.72
...          ...   ...   ...    ...     ...    ...    ...    ...
58180         20  2024    17  0.305  297.61  70.08  19.57  44.86
58181         20  2024    18  0.321  298.34  68.49  29.65  49.10
58182         20  2024    19  0.332  298.89  65.60  39.63  52.66
58183         20  2024    20  0.339  299.21  62.45  47.38  54.95
58184         20  2024    21  0.342  299.32  60.94  52.99  57.00

[53875 rows x 8 columns]


In [106]:
def region_year_analysis(region_id, years =("1982","2024"), ):

    df2 = resultdf[(resultdf["Year"].between(years[0],years[1])) & (resultdf['region_id'] == region_id)]
    # print(df2)
    region_name = reg_id_name[region_id]
    vhi_max_reg = df2["VHI"].max()
    vhi_min_reg = df2["VHI"].min()
    print(f"[+] {region_name}: min {vhi_min_reg} max {vhi_max_reg}")



In [107]:

# years = ("2000","2000")
for i in range(1,26):
    region_year_analysis(i)


[+] Вінницька: min 11.25 max 82.64
[+] Волинська: min 11.91 max 78.32
[+] Дніпропетровська: min 17.58 max 93.17
[+] Донецька: min 6.26 max 96.18
[+] Житомирська: min 19.48 max 77.57
[+] Закарпатська: min 10.88 max 96.69
[+] Запорізька: min nan max nan
[+] Івано-Франківська: min 18.98 max 73.35
[+] Київська: min 10.6 max 80.88
[+] Кіровоградська: min 16.36 max 84.52
[+] Луганська: min 12.45 max 90.32
[+] Львівська: min 18.31 max 69.96
[+] Миколаївська: min 5.94 max 92.31
[+] Одеська: min 5.52 max 89.14
[+] Полтавська: min 15.68 max 85.14
[+] Рівенська: min 20.7 max 77.45
[+] Сумська: min 12.64 max 81.96
[+] Тернопільська: min 19.45 max 77.71
[+] Харківська: min 9.36 max 91.42
[+] Херсонська: min 12.23 max 90.61
[+] Хмельницька: min 18.41 max 79.4
[+] Черкаська: min 10.68 max 83.7
[+] Чернівецька: min 15.16 max 74.92
[+] Чернігівська: min 15.17 max 80.65
[+] Республіка Крим: min 13.28 max 90.96


In [116]:
def drought_years(df, percentage, type_of_drought, start_year=None, end_year=None):
    # якщо рік не заданий - ставлю початковий/останній рік датафрейму
    if start_year is not None:
        df = df[df['Year'] >= start_year]
    if end_year is not None:
        df = df[df['Year'] <= end_year]

    # словник кількості областей з VHI менше порогового значення для кожного року
    drought_count = {}

    # групую дані за роками
    grouped_data = df.groupby('Year')

    # перебираю кожний рік
    for year, group_df in grouped_data:
        # знаходжу к-сть областей у цьому році
        total_areas = len(group_df['region_id'].unique())

        if type_of_drought == "extreme":
            # к-сть областей з VHI < 15
            drought_areas = group_df[group_df['VHI'] < 15]['region_id'].nunique()
        elif type_of_drought == "moderate":
            # к-сть областей з VHI > 15 але < 35
            drought_areas = group_df[(group_df['VHI'] > 15) & (group_df['VHI'] < 35)]['region_id'].nunique()
        
        # якщо к-сть посушливих областей > заданий відсоток - то зберігаю рік
        if drought_areas / total_areas > percentage / 100:
            drought_count[year] = (drought_areas, round((drought_areas / total_areas) * 100, 2))

    # створюю таблицю за допомогою PrettyTable
    table = PrettyTable()
    if type_of_drought == "extreme":
        table.title = f"Роки з екстремальною посухою (де більше {percentage}% областей мають VHI < 15)"
    elif type_of_drought == "moderate":
        table.title = f"Роки з помірною посухою (де більше {percentage}% областей мають VHI від 15 до 35)"

    table.field_names = ["Year", "К-сть областей з посухою", "Відсоток"]

    for year, (drought_areas, percent) in drought_count.items():
        table.add_row([year, drought_areas, f"{percent}%"])

    return table

# приклад для екстримальних
drought_years_extreme = drought_years(resultdf, 10, "extreme", start_year='2000', end_year='2024')
print(drought_years_extreme)

# приклад для помірних
drought_years_moderate = drought_years(resultdf, 10, "moderate", start_year='2000', end_year='2024')
print(drought_years_moderate)


+----------------------------------------------------------------------+
| Роки з екстремальною посухою (де більше 10% областей мають VHI < 15) |
+--------+-------------------------------------------+-----------------+
|  Year  |          К-сть областей з посухою         |     Відсоток    |
+--------+-------------------------------------------+-----------------+
|  2000  |                     4                     |      16.67%     |
|  2007  |                     5                     |      20.83%     |
+--------+-------------------------------------------+-----------------+
+-------------------------------------------------------------------------+
| Роки з помірною посухою (де більше 10% областей мають VHI від 15 до 35) |
+---------+---------------------------------------------+-----------------+
|   Year  |           К-сть областей з посухою          |     Відсоток    |
+---------+---------------------------------------------+-----------------+
|   2000  |                      24 